In [17]:
from sentence_transformers import SentenceTransformer
import pandas as pd

model = SentenceTransformer("jhgan/ko-sbert-nli")  # 한글 리뷰에 적합

# 파일명 수정해주세용
df = pd.read_csv('labeled/oliveyoung_lotion_labels.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234981 entries, 0 to 234980
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   제품명     234981 non-null  object
 1   성분      234981 non-null  object
 2   별점      234981 non-null  int64 
 3   피부타입    234981 non-null  object
 4   피부고민    234981 non-null  object
 5   자극도     234981 non-null  object
 6   카테고리    234981 non-null  object
 7   리뷰      234981 non-null  object
 8   리뷰_감성   234981 non-null  object
dtypes: int64(1), object(8)
memory usage: 16.1+ MB


In [ ]:
# 유저 질문 분석 함수
def extract_fields_from_question(question: str) -> list:
    field_keywords = {
        "제품명": ["제품명", "이름", "상품", "제품"],
        "카테고리": ["카테고리", "스킨", "토너", "에센스" ,"세럼", "앰플", "크림", "로션", "미스트", "오일"],
        "성분": ["성분", "히알루론산", "비타민", "AHA", "BHA", "나이아신아마이드"],
        "피부타입": ["건성", "지성", "복합성", "수부지", "수분부족지성", "극지성", "극건성", "속건조", "피부타입"],
        "피부고민": ["보습", "주름", "여드름", "미백", "흉터", "진정", "잡티", "흔적", "피부 고민"],
        "감정": ["좋았어요", "최악", "후회", "별점", "만족", "불만", "감정", "긍정", "부정", "중립"],
        "제형": ["제형", "찐득", "흡수", "쫀득", "미끌", "텍스처", "끈적", "마무리감", "마무리", "부드럽다"],
        "자극도": ["순해요", "트러블", "자극", "뾰루지", "뒤집어짐"]
    }
    result = []
    for field, keywords in field_keywords.items():
        if any(kw in question for kw in keywords):
            result.append(field)
    return result if result else ["제품명"]

In [ ]:
# from langchain.schema import Document
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# from langchain.chains import RetrievalQA
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough

# # 메타데이터 (원하는 컬럼만 선택)
# meta_columns = ["제품명", "성분", "별점", "피부타입", "피부고민", "자극도", "리뷰_감성"]

# # Document 리스트 생성
# documents = [
#     Document(
#         page_content=row["리뷰"],  # 리뷰 본문을 page_content로
#         metadata={col: row[col] for col in meta_columns}  # 필요한 메타데이터만
#     )
#     for _, row in df.iterrows()
# ]

# # LangChain HuggingFace 임베딩 객체 (다시 이걸로 임베딩)
# embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")

# # Chroma DB 생성 및 저장
# db = Chroma.from_documents(
#     documents=documents,
#     embedding=embedding,
#     persist_directory="chroma_oliveyoung"
# )

In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import uuid

In [21]:
# 1. 새 파일 불러오기
new_df = pd.read_csv("labeled/oliveyoung_lotion_labels.csv")  # ← 여기서 새 파일 불러오기

# 2. 필요한 컬럼만 추출
meta_columns = ["제품명", "성분", "별점", "피부타입", "피부고민", "자극도", "리뷰_감성"]

# 3. Document 리스트 생성
documents_new = [
    Document(
        page_content=row["리뷰"],  # 본문 텍스트
        metadata={col: row[col] for col in meta_columns}
    )
    for _, row in new_df.iterrows()
]

# 4. 기존 Chroma DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")
db = Chroma(
    persist_directory="./chroma_oliveyoung_reviews",
    embedding_function=embedding_model
)

# 5. 벡터 DB에 추가
db.add_documents(
    documents=documents_new,
    ids=[str(uuid.uuid4()) for _ in documents_new]
)

InternalError: ValueError: Batch size of 234981 is greater than max batch size of 5461

In [5]:
print(f"저장된 문서 개수: {db._collection.count()}")

저장된 문서 개수: 119556


In [8]:
# test
results = db.similarity_search("피부가 너무 건조한데, 미스트 제품 추천해줘", k=3)

for i, doc in enumerate(results, 1):
    print(f"--- Result {i} ---")
    print("내용:", doc.page_content)
    print("메타데이터:", doc.metadata)

--- Result 1 ---
내용: 요즘 피부가 너무 건조해서 미스트 구매했아요
메타데이터: {'리뷰_감성': '중립', '성분': '버지니아풍년화추출물,부틸렌글라이콜,다이프로필렌글라이콜,글리세린,폴리글리세릴-10라우레이트, 폴리글리세릴-10미리스테이트,카프릴릴글라이콜,1,2-헥산다이올,스타아니스추출물,라벤더오일,다이 소듐이디티에이,다이포타슘글리시리제이트,알란토인,자일리톨,베타인,병풀추출물,쇠비름추출물,뽕 나무껍질추출물,시트릭애씨드,리날룰', '자극도': '보통이에요', '피부고민': '보습에 좋아요', '피부타입': '복합성에 좋아요', '제품명': '그라운드플랜 미스트', '별점': 5}
--- Result 2 ---
내용: 요즘같이 피부가 너무 건조할때 뿌리기 좋은 미스트에요
메타데이터: {'별점': 5, '제품명': '그라운드플랜 미스트', '성분': '버지니아풍년화추출물,부틸렌글라이콜,다이프로필렌글라이콜,글리세린,폴리글리세릴-10라우레이트, 폴리글리세릴-10미리스테이트,카프릴릴글라이콜,1,2-헥산다이올,스타아니스추출물,라벤더오일,다이 소듐이디티에이,다이포타슘글리시리제이트,알란토인,자일리톨,베타인,병풀추출물,쇠비름추출물,뽕 나무껍질추출물,시트릭애씨드,리날룰', '피부고민': '보습에 좋아요', '리뷰_감성': '긍정', '피부타입': '건성에 좋아요', '자극도': '자극없이 순해요'}
--- Result 3 ---
내용: 피부가 너무 건조해서 미스트를 하나 구매했어요.
메타데이터: {'제품명': '메디톡스 뉴라덤 카밍 오로라 미스트', '별점': 5, '자극도': '자극없이 순해요', '성분': '정제수, 다이프로필렌글라이콜, 글리세린, 1,2-헥산다이올, 부틸렌글라이콜, 판테놀, 하이드로제네이티드레시틴, 세테아릴알코올, 솔비탄스테아레이트, 글리세릴스테아레이트, 자일리톨, 세라마이드엔피, 쇠비름추출물, 아데노신, 향료, 포스페이트버퍼드셀라인, 자일리틸글루코사이드, 안하이드로자일리톨, 인카르나타시계꽃전초추출물, 아르니카 몬타나꽃추출물, 참당귀뿌리추출물

In [ ]:
# LLM 설정
llm = ChatOpenAI(temperature=0.7, model="gpt-4")

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_template("""
      당신은 올리브영 스킨케어 화장품 정보를 전문적으로 안내하는 AI 어시스턴트입니다.
      사용자 질문에 따라 카테고리, 성분, 피부타입, 제형, 자극도, 감정 정보 등을 바탕으로 정확한 화장품 정보를 제공합니다.
      사용자의 피부 고민과 선호에 맞춰 카테고리에 맞는 화장품을 2~3 가지 추천하고, 관련 리뷰 정보를 요약해주는 것이 주요 역할입니다.

      # Instruction(지켜야 할 규칙):
      1. 반드시 제공된 문서(context)의 정보만을 기반으로 답변하세요. 
         문서에 없는 사실을 추측하거나 임의로 생성하지 마세요.
      2. 화장품의 이름과 카테고리는 반드시 '카테고리'와 '제품명' 필드를 참고하여 확인하세요. 
         카테고리는 스킨/토너, 에센스/세럼/앰플, 크림, 로션, 미스트/오일로 구성되어 있습니다.
      3. 가능한 한 명확하고 간결하게 답변하세요.
      4. 화장품 정보를 안내할 때는 다음 순서를 지키세요: 
         - 제품명
         - 주요 성분 (가능한 경우)
         - 평점 또는 긍정 리뷰 요약
      5. 문장 스타일은 전문성과 친근함을 겸비한 대화체로 작성하세요. 
         예: “이 제품은 복합성 피부에 정말 잘 맞는다고 해요!”
      6. “문서에 따르면”, “문맥에서 보면”과 같은 표현은 사용하지 마세요. 자연스럽게 설명만 하세요.
      7. 질문이 모호하거나 정보가 부족할 경우, 필요한 정보를 정중하게 요청하세요.
      8. 출력은 보기 쉽게 줄바꿈을 해서 전달해주세요.
      9. 제품 추천 및 설명 마지막에 특징을 잘 나타내는 문서(별점은 제외)들을 세 개 정도 출력해주세요.
                                                   
      # Context(문서 요약 정보):
      {context}

      # 질문:
      {question}
      """)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = db.as_retriever()
# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
# 예시 질문
question = "속건조에 좋은 미스트 3개만 알려주세요."
response = rag_chain.invoke(question)

print("🧠 응답:")
print(response)

🧠 응답:
차앤박(CNP) 프로폴리스 에너지 앰플 미스트는 건성 피부에게 꽤 좋은 평가를 받았습니다. 

주요 성분으로는 프로폴리스 추출물과 하이알루로닉애씨드가 들어있는데, 이런 성분들은 피부에 충분한 보습을 제공하며, 피부를 유연하게 유지하는데 도움을 줍니다. 

이 제품에 대한 사용 후기는 '자극 없이 순하다'며 피부에 부드럽게 적용되고 보습에 효과적이라는 평가를 받았습니다. 별점은 5점 만점에 5점을 받았습니다. 

이 제품에 대한 다른 사용 후기를 확인하시려면 아래의 문서를 참고하세요: 
- Document ID: '0d5e895b-72b0-46eb-8b05-3541b04707e8' 
- Document ID: '018a13d4-0f21-45dc-bd4c-5fc2009e5a29' 
- Document ID: '7b55fc4b-512b-4d9f-93b1-e687831a3094' 
- Document ID: '49db2645-2b14-4ea8-8e99-1c17bf794143'
